In [1]:
# default_exp rank

In [1]:
%reload_ext autoreload
%autoreload 2

%matplotlib inline

In [2]:
# export
import os
import sys
sys.path.append('../mylib/')
from code.eda import * 
from code.config import args
import pandas as pd  
import numpy as np
from tqdm import tqdm  
from loguru import logger
from utils.pickle import PickleWrapper

In [3]:
# export
trace = logger.add(os.path.join(args.DATA_DIR, 'data_gen/runtime.log'))  

# val

In [4]:
# export
def val1(df, topk, click_val):
    """
    """
    sr = df.groupby('user_id')['item_id_pred'].agg(list)
    r_list = []
    pred_num = click_val.shape[0]
    for line in click_val.itertuples():
        try:
            df.loc[df.user_id==line.user_id, 'item_id_true'] = line.item_id
            t = sr[line.user_id]
            i = t.index(line.item_id)
        except:
            i = 99999

        r_list.append(i)
    r_series = pd.Series(r_list)
    for i in range(50, topk+1, 50):
        logger.info(f'recall{i}:{r_series[r_series<i].shape[0]/pred_num}')
        
#     r_series[r_series<9999].plot(kind='box', title='pp', rot=45)
    return df

# load click data

In [5]:
# export
all_click_df = load_click_data_per_phase(args.now_phase, args.DATA_DIR)

2020-06-03 05:48:26.279 | INFO     | code.eda:load_click_data_per_phase:97 - phase: 0
2020-06-03 05:48:27.446 | INFO     | code.eda:load_click_data_per_phase:108 - all_click: (261337, 3), click_test1_val: (1663, 3), test_qtime1: (1663, 2)
2020-06-03 05:48:27.449 | INFO     | code.eda:load_click_data_per_phase:97 - phase: 1
2020-06-03 05:48:28.625 | INFO     | code.eda:load_click_data_per_phase:108 - all_click: (264871, 3), click_test1_val: (1726, 3), test_qtime1: (1726, 2)
2020-06-03 05:48:28.629 | INFO     | code.eda:load_click_data_per_phase:97 - phase: 2
2020-06-03 05:48:29.871 | INFO     | code.eda:load_click_data_per_phase:108 - all_click: (264624, 3), click_test1_val: (1690, 3), test_qtime1: (1690, 2)
2020-06-03 05:48:29.873 | INFO     | code.eda:load_click_data_per_phase:97 - phase: 3
2020-06-03 05:48:31.024 | INFO     | code.eda:load_click_data_per_phase:108 - all_click: (286609, 3), click_test1_val: (1675, 3), test_qtime1: (1675, 2)
2020-06-03 05:48:31.025 | INFO     | code.ed

In [6]:
# export
item_feat = get_item_data()

/root/data1/myappsexlib/a_0330_0527_Modern_E_Commerce_Platform_Debiasing/code/eda.py:32: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  converters={'item_id':str})


In [208]:
item_feat.iloc[:, 1:129].shape

(108916, 128)

In [209]:
item_feat.iloc[:, 129:257].shape

(108916, 128)

In [7]:
# export
item_feat['text_vec'] = item_feat.iloc[:, 1:129].values.tolist()
item_feat['img_vec'] = item_feat.iloc[:, 129:257].values.tolist()

item_feat['text_vec'] = item_feat['text_vec'].map(np.array)

item_feat['img_vec'] = item_feat['img_vec'].map(np.array)

item_feat.set_index('item_id', inplace=True)

In [48]:
# export
phase = [0, 1, 2, 3, 5, 6, 4]

# phase = [4]
# match_num = 200
match_num = 500

In [185]:
# 计算每个phase的item_id user_cnt  # 貌似每个phase的item只会被user唯一点击一次，那就没必要统计了。。。
def t(s):
    return s.nunique()
all_click_df[0][0].groupby('item_id')['user_id'].agg(t)

item_id
1          4
1000      13
100000     8
10001      5
100013     5
          ..
99990      6
99991      3
99993      5
99994      5
99998      5
Name: user_id, Length: 40775, dtype: int64

In [177]:
# temp_ = all_click_df[0][0].groupby('user_id')['item_id'].agg(list)
temp_ = all_click_df[0][0].groupby('item_id')['user_id'].agg(list)

In [179]:
temp_

item_id
1                               [1489, 10088, 13420, 19312]
1000      [14546, 9531, 32114, 6934, 20558, 30360, 33492...
100000    [20800, 17064, 33901, 5699, 8541, 1940, 20705,...
10001                      [6572, 17178, 25162, 6383, 8264]
100013                    [18924, 32480, 7696, 6319, 32706]
                                ...                        
99990              [30399, 8084, 2969, 21225, 32152, 34183]
99991                                  [9430, 32852, 31855]
99993                   [23622, 22023, 12546, 31661, 21784]
99994                     [9287, 6305, 28964, 17648, 22420]
99998                    [24217, 9796, 20948, 25551, 16599]
Name: user_id, Length: 40775, dtype: object

In [178]:
temp_.str.len().describe()

count    40775.000000
mean         6.409246
std          6.358793
min          1.000000
25%          4.000000
50%          5.000000
75%          7.000000
max        234.000000
Name: user_id, dtype: float64

In [ ]:
for p in phase:
    # 获取test用户的近n次点击
    temp_ = all_click_df[0][0].groupby('user_id')['item_id'].agg(list)
    temp_ = pd.DataFrame(temp_)
    for i in range(5):
        temp_[f'last_{i+1}'] = temp_.item_id.str.get(-(i+1))
        
    # 计算每个phase的item_id cnt
    all_click_df[0][0].groupby('item_id')['user_id'].count()

# load match result

In [49]:
# export
def load_match_items():
    r_itemcf = PickleWrapper.loadFromFile(os.path.join(args.DATA_DIR, 'data_gen/r_list_itemcf_0527.pkl'))

    r_binn = PickleWrapper.loadFromFile(os.path.join(args.DATA_DIR, 'data_gen/r_list_binn_0527.pkl'))
    
    r_itemcf_phase4 = PickleWrapper.loadFromFile(os.path.join(args.DATA_DIR, 'data_gen/r_list_itemcf_0527_phase4.pkl'))

    r_binn_phase4 = PickleWrapper.loadFromFile(os.path.join(args.DATA_DIR, 'data_gen/r_list_binn_0527_phase4.pkl'))

    
    r_itemcf_yl = PickleWrapper.loadFromFile(os.path.join(args.DATA_DIR, 'data_gen/r_list_itemcf_yulao_0527.pkl'))
    r_list_w2v = PickleWrapper.loadFromFile(os.path.join(args.DATA_DIR, 'data_gen/r_list_w2v_0602_all.pkl'))
    r_itemcf = r_itemcf + r_itemcf_phase4
    r_binn = r_binn + r_binn_phase4
    r_itemcf_yl = [r_itemcf_yl[0], r_itemcf_yl[1],r_itemcf_yl[2],r_itemcf_yl[3],r_itemcf_yl[5],r_itemcf_yl[6],r_itemcf_yl[4]]
    
    r_list_w2v = [r_list_w2v[0], r_list_w2v[1],r_list_w2v[2],r_list_w2v[3],r_list_w2v[5],r_list_w2v[6],r_list_w2v[4]]
    return r_itemcf, r_binn, r_itemcf_yl, r_list_w2v
#     return r_itemcf_phase4, r_binn_phase4, [r_itemcf_yl[4]]


In [50]:
r_itemcf, r_binn, r_itemcf_yl, r_list_w2v = load_match_items()

In [119]:
# export

def data_preporcess(recall_list, match_num, phase, mode='train'):
    
    itemcf, r_binn, r_itemcf_yl, r_list_w2v = recall_list
    df1 = pd.DataFrame()
    df2 = pd.DataFrame()
    df3 = pd.DataFrame()
    df4 = pd.DataFrame()
    for i in range(len(phase)):
        r_itemcf[i]['phase'] = phase[i]
        r_binn[i]['phase'] = phase[i]
        r_itemcf_yl[i]['phase'] = phase[i]
        r_list_w2v[i]['phase'] = phase[i]
        df1 = df1.append(r_itemcf[i])
        df2 = df2.append(r_binn[i])
        df3 = df3.append(r_itemcf_yl[i])
        df4 = df4.append(r_list_w2v[i])

    df1_ = df1[df1['rank']<(match_num+1)]
    df2_ = df2[df2['rank']<(match_num+1)]
    df3_ = df3[df3['rank']<(match_num+1)]
    df4_ = df4[df4['rank']<(match_num+1)]

    print('merge Multi-channel recall...')
    cols = 'user_id item_id_pred phase'.split()
    
    if mode == 'train':
        cols = 'user_id item_id_pred item_id_true phase'.split()
    df = pd.merge(pd.merge(pd.merge(df1_, 
                                    df2_, on=cols, how='outer'), 
                                      df3_, on=cols, how='outer'), 
                                      df4_, on=cols, how='outer')

    df = df.fillna(0)
    if mode == 'train':
        df['label'] = (df['item_id_pred'] == df['item_id_true']).map(int)


    dft = pd.DataFrame()
    for p in tqdm(phase):
        # 获取test用户的近n次点击
        temp_ = all_click_df[p][0][all_click_df[p][0].user_id.isin(all_click_df[p][1].user_id)].groupby('user_id')['item_id'].agg(list)
        temp_ = pd.DataFrame(temp_)
        for i in range(0, 3):
            temp_[f'last_{i+1}'] = temp_.item_id.str.get(-(i+1))
        dft = dft.append(temp_.reset_index())
        # 计算每个phase的item_id cnt
        t_ = all_click_df[p][0].groupby('item_id')['user_id'].count()
        df.loc[df.phase==p, 'item_cnt'] = df.item_id_pred.map(lambda x: t_[x] if x in t_ else 0)

    df = pd.merge(df, dft.drop(columns='item_id'))

    

    # 太大了，不建议存储    
    # df.to_pickle(os.path.join(args.DATA_DIR, f'data_gen/dft{match_num}.pkl'))

    # df = pd.read_pickle(os.path.join(args.DATA_DIR, f'data_gen/dft{match_num}.pkl'))
    return df

In [52]:
recall_list = [r_itemcf, r_binn, r_itemcf_yl, r_list_w2v]
df = data_preporcess(recall_list, match_num, phase)

merge Multi-channel recall...




  0%|          | 0/7 [00:00<?, ?it/s]

 14%|█▍        | 1/7 [02:09<12:57, 129.64s/it]

 29%|██▊       | 2/7 [04:18<10:46, 129.39s/it]

 43%|████▎     | 3/7 [06:35<08:47, 131.82s/it]

 57%|█████▋    | 4/7 [08:51<06:39, 133.07s/it]

 71%|███████▏  | 5/7 [11:03<04:25, 132.54s/it]

 86%|████████▌ | 6/7 [13:10<02:10, 130.94s/it]

100%|██████████| 7/7 [15:17<00:00, 131.09s/it]


In [53]:
df.head()

,user_id,item_id_pred,score_x,rank_x,item_id_true,phase,score_y,rank_y,score_x,rank_x,score_y,rank_y,label,item_cnt,last_1,last_2,last_3
0,11,79868,6.019197,1.0,8052,0,2.023002,1.0,0.010625,142.0,0.000000,0.0,0,47.0,28210,64689,20317
1,11,15844,5.328522,2.0,8052,0,0.326349,86.0,0.086894,6.0,23.022548,9.0,0,5.0,28210,64689,20317
2,11,10528,5.246875,3.0,8052,0,0.461508,52.0,0.067450,11.0,19.745578,23.0,0,7.0,28210,64689,20317
3,11,113564,5.100132,4.0,8052,0,0.386057,66.0,0.136157,1.0,0.000000,0.0,0,4.0,28210,64689,20317
4,11,20389,4.636769,5.0,8052,0,0.415892,62.0,0.048790,19.0,0.000000,0.0,0,7.0,28210,64689,20317


In [54]:
df.columns

Index(['user_id', 'item_id_pred', 'score_x', 'rank_x', 'item_id_true', 'phase',
       'score_y', 'rank_y', 'score_x', 'rank_x', 'score_y', 'rank_y', 'label',
       'item_cnt', 'last_1', 'last_2', 'last_3'],
      dtype='object')

In [55]:
df.to_pickle(os.path.join(args.DATA_DIR, f'data_gen/dft{match_num}_w2v_data_preporcess.pkl'))

In [58]:
df.shape

(15585787, 17)

In [17]:
temp_ = pd.read_pickle(os.path.join(args.DATA_DIR, f'data_gen/dft{match_num}_features.pkl'))

temp4_ = pd.read_pickle(os.path.join(args.DATA_DIR, f'data_gen/dft{match_num}_features_phase4.pkl'))

In [19]:
temp_.shape

(9535777, 26)

In [20]:
temp4_.shape

(1681258, 58)

In [68]:
temp_.columns

Index(['user_id', 'item_id_pred', 'score_x', 'rank_x', 'item_id_true', 'phase',
       'score_y', 'rank_y', 'score', 'rank', 'label', 'item_cnt', 'item_id',
       'last_1', 'last_2', 'last_3', 'sim1_text', 'sim1_img', 'sim2_text',
       'sim2_img', 'sim3_text', 'sim3_img', 'sim4_text', 'sim4_img',
       'sim5_text', 'sim5_img'],
      dtype='object')

In [70]:
df.columns = ['user_id', 'item_id_pred', 'score_1', 'rank_1', 'item_id_true', 'phase',
       'score_2', 'rank_2', 'score_3', 'rank_3', 'score_4', 'rank_4', 'label',
       'item_cnt', 'last_1', 'last_2', 'last_3']

In [81]:
df['t'] = range(df.shape[0])

In [82]:
df_ = pd.merge(df, temp_[['user_id', 'item_id_pred', 'item_id_true', 'phase', 'sim1_text', 'sim1_img', 'sim2_text',
       'sim2_img', 'sim3_text', 'sim3_img']], on=['user_id', 'item_id_pred', 'item_id_true', 'phase'])

In [83]:
df_4 = pd.merge(df, temp4_[['user_id', 'item_id_pred', 'item_id_true', 'phase', 'sim1_text', 'sim1_img', 'sim2_text',
       'sim2_img', 'sim3_text', 'sim3_img']], on=['user_id', 'item_id_pred', 'item_id_true', 'phase'])

In [84]:
df1 = df[~df.t.isin(df_.t)]

In [86]:
df1 = df1[~df1.t.isin(df_4.t)]

In [87]:
df1.shape

(4368752, 18)

In [88]:
df1.columns

Index(['user_id', 'item_id_pred', 'score_1', 'rank_1', 'item_id_true', 'phase',
       'score_2', 'rank_2', 'score_3', 'rank_3', 'score_4', 'rank_4', 'label',
       'item_cnt', 'last_1', 'last_2', 'last_3', 't'],
      dtype='object')

In [89]:
df1['item_id_pred_text_vec'] = df1.item_id_pred.map(lambda x: item_feat['text_vec'][x] if x in item_feat['text_vec'] else np.zeros(128))

df1['item_id_pred_img_vec'] = df1.item_id_pred.map(lambda x: item_feat['img_vec'][x] if x in item_feat['img_vec'] else np.arange(128))

for c in tqdm('last_1 last_2 last_3'.split()):
    df1[f'{c}_text_vec'] = df1.item_id_pred.map(lambda x: item_feat['text_vec'][x] if x in item_feat['text_vec'] else np.zeros(128))

    df1[f'{c}_img_vec'] = df1.item_id_pred.map(lambda x: item_feat['img_vec'][x] if x in item_feat['img_vec'] else np.arange(128))




  0%|          | 0/3 [00:00<?, ?it/s]

 33%|███▎      | 1/3 [03:17<06:34, 197.11s/it]

 67%|██████▋   | 2/3 [06:31<03:16, 196.30s/it]

100%|██████████| 3/3 [09:47<00:00, 195.75s/it]


In [90]:
df1['sim1_text'] = (df1['item_id_pred_text_vec'] * df1['last_1_text_vec']).map(sum)

df1['sim1_img'] = (df1['item_id_pred_img_vec'] * df1['last_1_img_vec']).map(sum)

df1['sim2_text'] = (df1['item_id_pred_text_vec'] * df1['last_2_text_vec']).map(sum)
df1['sim2_img'] = (df1['item_id_pred_img_vec'] * df1['last_2_img_vec']).map(sum)

df1['sim3_text'] = (df1['item_id_pred_text_vec'] * df1['last_3_text_vec']).map(sum)
df1['sim3_img'] = (df1['item_id_pred_img_vec'] * df1['last_3_img_vec']).map(sum)

In [91]:
df1.shape

(4368752, 32)

In [92]:
df1.columns

Index(['user_id', 'item_id_pred', 'score_1', 'rank_1', 'item_id_true', 'phase',
       'score_2', 'rank_2', 'score_3', 'rank_3', 'score_4', 'rank_4', 'label',
       'item_cnt', 'last_1', 'last_2', 'last_3', 't', 'item_id_pred_text_vec',
       'item_id_pred_img_vec', 'last_1_text_vec', 'last_1_img_vec',
       'last_2_text_vec', 'last_2_img_vec', 'last_3_text_vec',
       'last_3_img_vec', 'sim1_text', 'sim1_img', 'sim2_text', 'sim2_img',
       'sim3_text', 'sim3_img'],
      dtype='object')

In [93]:
df_.columns

Index(['user_id', 'item_id_pred', 'score_1', 'rank_1', 'item_id_true', 'phase',
       'score_2', 'rank_2', 'score_3', 'rank_3', 'score_4', 'rank_4', 'label',
       'item_cnt', 'last_1', 'last_2', 'last_3', 't', 'sim1_text', 'sim1_img',
       'sim2_text', 'sim2_img', 'sim3_text', 'sim3_img'],
      dtype='object')

In [94]:
df_4.columns

Index(['user_id', 'item_id_pred', 'score_1', 'rank_1', 'item_id_true', 'phase',
       'score_2', 'rank_2', 'score_3', 'rank_3', 'score_4', 'rank_4', 'label',
       'item_cnt', 'last_1', 'last_2', 'last_3', 't', 'sim1_text', 'sim1_img',
       'sim2_text', 'sim2_img', 'sim3_text', 'sim3_img'],
      dtype='object')

In [96]:
dff = df1[df_.columns].append(df_4).append(df_)

In [97]:
dff.shape

(15585787, 24)

In [53]:
# df.to_pickle(os.path.join(args.DATA_DIR, f'data_gen/dft{match_num}_features_phase4.pkl'))

In [27]:
# sim_text_dict = {}
# sim_img_dict = {}

# FE

In [ ]:
# export
def fe(df):
    for i in tqdm('rank score'.split()):
        df[f'{i}1_sub_{i}2'] = df[f'{i}_1']-df[f'{i}_2']
        df[f'{i}1_add_{i}2'] = df[f'{i}_1']+df[f'{i}_2']
        df[f'{i}1_mul_{i}2'] = df[f'{i}_1']*df[f'{i}_2']

        df[f'{i}1_sub_{i}3'] = df[f'{i}_1']-df[f'{i}_3']
        df[f'{i}1_add_{i}3'] = df[f'{i}_1']+df[f'{i}_3']
        df[f'{i}1_mul_{i}3'] = df[f'{i}_1']*df[f'{i}_3']

        df[f'{i}1_sub_{i}4'] = df[f'{i}_1']-df[f'{i}_4']
        df[f'{i}1_add_{i}4'] = df[f'{i}_1']+df[f'{i}_4']
        df[f'{i}1_mul_{i}4'] = df[f'{i}_1']*df[f'{i}_4']

        df[f'{i}2_sub_{i}3'] = df[f'{i}_2']-df[f'{i}_3']
        df[f'{i}2_add_{i}3'] = df[f'{i}_2']+df[f'{i}_3']
        df[f'{i}2_mul_{i}3'] = df[f'{i}_2']*df[f'{i}_3']

        df[f'{i}2_sub_{i}4'] = df[f'{i}_2']-df[f'{i}_4']
        df[f'{i}2_add_{i}4'] = df[f'{i}_2']+df[f'{i}_4']
        df[f'{i}2_mul_{i}4'] = df[f'{i}_2']*df[f'{i}_4']

        df[f'{i}3_sub_{i}4'] = df[f'{i}_3']-df[f'{i}_4']
        df[f'{i}3_add_{i}4'] = df[f'{i}_3']+df[f'{i}_4']
        df[f'{i}3_mul_{i}4'] = df[f'{i}_3']*df[f'{i}_4']
        
#     df['sim1_text'] = (df['item_id_pred_text_vec'] * df['last_1_text_vec']).map(sum)

#     df['sim1_img'] = (df['item_id_pred_img_vec'] * df['last_1_img_vec']).map(sum)

#     df['sim2_text'] = (df['item_id_pred_text_vec'] * df['last_2_text_vec']).map(sum)
#     df['sim2_img'] = (df['item_id_pred_img_vec'] * df['last_2_img_vec']).map(sum)

#     df['sim3_text'] = (df['item_id_pred_text_vec'] * df['last_3_text_vec']).map(sum)
#     df['sim3_img'] = (df['item_id_pred_img_vec'] * df['last_3_img_vec']).map(sum)

#     df['sim4_text'] = (df['item_id_pred_text_vec'] * df['last_4_text_vec']).map(sum)
#     df['sim4_img'] = (df['item_id_pred_img_vec'] * df['last_4_img_vec']).map(sum)

#     df['sim5_text'] = (df['item_id_pred_text_vec'] * df['last_5_text_vec']).map(sum)
#     df['sim5_img'] = (df['item_id_pred_img_vec'] * df['last_5_img_vec']).map(sum)
    
    

    df['sim1_text_img'] = df['sim1_text'] *  df['sim1_img']
    df['sim2_text_img'] = df['sim2_text'] *  df['sim2_img']
    df['sim3_text_img'] = df['sim3_text'] *  df['sim3_img']
#     df['sim4_text_img'] = df['sim4_text'] *  df['sim4_img']
#     df['sim5_text_img'] = df['sim5_text'] *  df['sim5_img']

    df['sim12_text'] = df['sim1_text'] + df['sim2_text']
    df['sim123_text'] = df['sim1_text'] + df['sim2_text'] + df['sim3_text']
#     df['sim1234_text'] = df['sim1_text'] + df['sim2_text'] + df['sim3_text'] + df['sim4_text']
#     df['sim12345_text'] = df['sim1_text'] + df['sim2_text'] + df['sim3_text'] + df['sim4_text'] + df['sim5_text']

    df['sim12_img'] = df['sim1_img'] + df['sim2_img']
    df['sim123_img'] = df['sim1_img'] + df['sim2_img'] + df['sim3_img']
#     df['sim1234_img'] = df['sim1_img'] + df['sim2_img'] + df['sim3_img'] + df['sim4_img']
#     df['sim12345_img'] = df['sim1_img'] + df['sim2_img'] + df['sim3_img'] + df['sim4_img'] + df['sim5_img']

    df['sim12_text_img'] = df['sim1_text_img'] + df['sim2_text_img']
    df['sim123_text_img'] = df['sim1_text_img'] + df['sim2_text_img'] + df['sim3_text_img']
#     df['sim1234_text_img'] = df['sim1_text_img'] + df['sim2_text_img'] + df['sim3_text_img'] + df['sim4_text_img']
#     df['sim12345_text_img'] = df['sim1_text_img'] + df['sim2_text_img'] + df['sim3_text_img'] + df['sim4_text_img'] + df['sim5_text_img']
 
    
    return df

In [99]:
dff = fe(dff)



  0%|          | 0/2 [00:00<?, ?it/s]

 50%|█████     | 1/2 [00:00<00:00,  2.37it/s]

100%|██████████| 2/2 [00:00<00:00,  2.38it/s]


In [100]:
dff.head()

,user_id,item_id_pred,score_1,rank_1,item_id_true,phase,score_2,rank_2,score_3,rank_3,score_4,rank_4,label,item_cnt,last_1,last_2,last_3,t,sim1_text,sim1_img,sim2_text,sim2_img,sim3_text,sim3_img,rank1_sub_rank2,rank1_add_rank2,rank1_mul_rank2,rank1_sub_rank3,rank1_add_rank3,rank1_mul_rank3,rank1_sub_rank4,rank1_add_rank4,rank1_mul_rank4,rank2_sub_rank3,rank2_add_rank3,rank2_mul_rank3,rank2_sub_rank4,rank2_add_rank4,rank2_mul_rank4,rank3_sub_rank4,rank3_add_rank4,rank3_mul_rank4,score1_sub_score2,score1_add_score2,score1_mul_score2,score1_sub_score3,score1_add_score3,score1_mul_score3,score1_sub_score4,score1_add_score4,score1_mul_score4,score2_sub_score3,score2_add_score3,score2_mul_score3,score2_sub_score4,score2_add_score4,score2_mul_score4,score3_sub_score4,score3_add_score4,score3_mul_score4,sim1_text_img,sim2_text_img,sim3_text_img,sim12_text,sim123_text,sim12_img,sim123_img,sim12_text_img,sim123_text_img
786,11,66987,0.0,0.0,8052,0,0.0,0.0,0.0,0.0,29.806313,1.0,0,4.0,28210,64689,20317,786,711.084763,557.705309,711.084763,557.705309,711.084763,557.705309,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,1.0,0.0,0.0,0.0,0.0,-1.0,1.0,0.0,-1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-29.806313,29.806313,0.0,0.0,0.0,0.0,-29.806313,29.806313,0.0,-29.806313,29.806313,0.0,396575.747305,396575.747305,396575.747305,1422.169526,2133.254289,1115.410618,1673.115926,7.931515e+05,1.189727e+06
787,11,94944,0.0,0.0,8052,0,0.0,0.0,0.0,0.0,27.631903,2.0,0,5.0,28210,64689,20317,787,1051.756525,652.120819,1051.756525,652.120819,1051.756525,652.120819,0.0,0.0,0.0,0.0,0.0,0.0,-2.0,2.0,0.0,0.0,0.0,0.0,-2.0,2.0,0.0,-2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-27.631903,27.631903,0.0,0.0,0.0,0.0,-27.631903,27.631903,0.0,-27.631903,27.631903,0.0,685872.325816,685872.325816,685872.325816,2103.513049,3155.269574,1304.241637,1956.362456,1.371745e+06,2.057617e+06
788,11,97323,0.0,0.0,8052,0,0.0,0.0,0.0,0.0,26.758751,3.0,0,3.0,28210,64689,20317,788,646.239106,355.463557,646.239106,355.463557,646.239106,355.463557,0.0,0.0,0.0,0.0,0.0,0.0,-3.0,3.0,0.0,0.0,0.0,0.0,-3.0,3.0,0.0,-3.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-26.758751,26.758751,0.0,0.0,0.0,0.0,-26.758751,26.758751,0.0,-26.758751,26.758751,0.0,229714.451160,229714.451160,229714.451160,1292.478212,1938.717318,710.927114,1066.390670,4.594289e+05,6.891434e+05
789,11,42055,0.0,0.0,8052,0,0.0,0.0,0.0,0.0,25.652910,4.0,0,5.0,28210,64689,20317,789,728.438766,510.922080,728.438766,510.922080,728.438766,510.922080,0.0,0.0,0.0,0.0,0.0,0.0,-4.0,4.0,0.0,0.0,0.0,0.0,-4.0,4.0,0.0,-4.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-25.652910,25.652910,0.0,0.0,0.0,0.0,-25.652910,25.652910,0.0,-25.652910,25.652910,0.0,372175.449728,372175.449728,372175.449728,1456.877533,2185.316299,1021.844160,1532.766240,7.443509e+05,1.116526e+06
790,11,74451,0.0,0.0,8052,0,0.0,0.0,0.0,0.0,25.053457,5.0,0,3.0,28210,64689,20317,790,810.390243,537.873010,810.390243,537.873010,810.390243,537.873010,0.0,0.0,0.0,0.0,0.0,0.0,-5.0,5.0,0.0,0.0,0.0,0.0,-5.0,5.0,0.0,-5.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-25.053457,25.053457,0.0,0.0,0.0,0.0,-25.053457,25.053457,0.0,-25.053457,25.053457,0.0,435887.039099,435887.039099,435887.039099,1620.780486,2431.170729,1075.746019,1613.619029,8.717741e+05,1.307661e+06


In [101]:
dff.shape

(15585787, 69)

In [141]:
df.shape

(9536065, 10)

In [144]:
df.label.value_counts()

0    9533232
1       2833
Name: label, dtype: int64

In [102]:
dff.label.value_counts()

0    15581708
1        4079
Name: label, dtype: int64

In [103]:
dff.user_id.nunique()

12080

In [104]:
dff.columns

Index(['user_id', 'item_id_pred', 'score_1', 'rank_1', 'item_id_true', 'phase',
       'score_2', 'rank_2', 'score_3', 'rank_3', 'score_4', 'rank_4', 'label',
       'item_cnt', 'last_1', 'last_2', 'last_3', 't', 'sim1_text', 'sim1_img',
       'sim2_text', 'sim2_img', 'sim3_text', 'sim3_img', 'rank1_sub_rank2',
       'rank1_add_rank2', 'rank1_mul_rank2', 'rank1_sub_rank3',
       'rank1_add_rank3', 'rank1_mul_rank3', 'rank1_sub_rank4',
       'rank1_add_rank4', 'rank1_mul_rank4', 'rank2_sub_rank3',
       'rank2_add_rank3', 'rank2_mul_rank3', 'rank2_sub_rank4',
       'rank2_add_rank4', 'rank2_mul_rank4', 'rank3_sub_rank4',
       'rank3_add_rank4', 'rank3_mul_rank4', 'score1_sub_score2',
       'score1_add_score2', 'score1_mul_score2', 'score1_sub_score3',
       'score1_add_score3', 'score1_mul_score3', 'score1_sub_score4',
       'score1_add_score4', 'score1_mul_score4', 'score2_sub_score3',
       'score2_add_score3', 'score2_mul_score3', 'score2_sub_score4',
       'score2_add_s

In [105]:
# dff.to_pickle(os.path.join(args.DATA_DIR, f'data_gen/dft{match_num}_train_features.pkl'))

In [157]:
dff = pd.read_pickle(os.path.join(args.DATA_DIR, f'data_gen/dft{match_num}_train_features.pkl'))

# model

In [175]:
#export
def get_model_input(df, mode='train'):
    from catboost import Pool, CatBoostClassifier

    cols = ['score_1', 'rank_1',
       'score_2', 'rank_2', 'score_3', 'rank_3', 'score_4', 'rank_4',
       'item_cnt', 'last_1', 'last_2', 'last_3', 'sim1_text', 'sim1_img',
       'sim2_text', 'sim2_img', 'sim3_text', 'sim3_img', 'rank1_sub_rank2',
       'rank1_add_rank2', 'rank1_mul_rank2', 'rank1_sub_rank3',
       'rank1_add_rank3', 'rank1_mul_rank3', 'rank1_sub_rank4',
       'rank1_add_rank4', 'rank1_mul_rank4', 'rank2_sub_rank3',
       'rank2_add_rank3', 'rank2_mul_rank3', 'rank2_sub_rank4',
       'rank2_add_rank4', 'rank2_mul_rank4', 'rank3_sub_rank4',
       'rank3_add_rank4', 'rank3_mul_rank4', 'score1_sub_score2',
       'score1_add_score2', 'score1_mul_score2', 'score1_sub_score3',
       'score1_add_score3', 'score1_mul_score3', 'score1_sub_score4',
       'score1_add_score4', 'score1_mul_score4', 'score2_sub_score3',
       'score2_add_score3', 'score2_mul_score3', 'score2_sub_score4',
       'score2_add_score4', 'score2_mul_score4', 'score3_sub_score4',
       'score3_add_score4', 'score3_mul_score4', 'sim1_text_img',
       'sim2_text_img', 'sim3_text_img', 'sim12_text', 'sim123_text',
       'sim12_img', 'sim123_img', 'sim12_text_img', 'sim123_text_img']
    df['fscore'] = 0
    df['frank'] = 999
    cat_cols = []
    if mode=='train':
        X_train, X_test, y_train, y_test = df.loc[df['phase']<6, cols], df.loc[df['phase']==6, cols], df.loc[df['phase']<6, 'label'],\
                                               df.loc[df['phase']==6, 'label']                 

        
        train_data = Pool(data=X_train,
                           label=y_train,
                           cat_features=cat_cols)
        val_data = Pool(data=X_test, label=y_test, cat_features=cat_cols) 
        return train_data, val_data, cat_cols
#     cols.remove('label')
    X_test = df[cols]
    test_data = Pool(data=X_test, cat_features=cat_cols) 
    return test_data, cat_cols

In [127]:
# df.loc[:, 'rank_x'] = df.rank_x.map(lambda x: 999 if x==0 else x)
# df.loc[:, 'rank_y'] = df.rank_x.map(lambda x: 999 if x==0 else x)
# df.loc[:, 'rank'] = df.rank_x.map(lambda x: 999 if x==0 else x)

In [159]:
train_data, val_data, cat_cols = get_model_input(dff)

In [108]:
dff['phase'].value_counts()

6    2481465
5    2432847
4    2293060
3    2204614
2    2179516
1    2149532
0    1844753
Name: phase, dtype: int64

In [109]:
dff.loc[df['phase']==6].user_id.nunique()

1821

In [110]:
dff.loc[dff['phase']==6].shape

(2481465, 71)

In [168]:
# export
def get_model():
    from catboost import Pool, CatBoostClassifier
    params = {
        'iterations': 140,
        'learning_rate': 0.1,
        'random_seed': 144,
        'custom_metric': 'F1',
        'loss_function': 'Logloss',
        'class_weights': [1, 20],
    #     'use_best_model': False
        }
    print(params)
    model = CatBoostClassifier(**params)
    return model

In [169]:
# val1(r_itemcf[-1], 100, all_click_df[6][1])
# 2020-05-27 17:39:59.717 | INFO     | __main__:val1:24 - recall50:0.15595826468973092
# 2020-05-27 17:39:59.728 | INFO     | __main__:val1:24 - recall100:0.18231740801757276


# fit

In [170]:
model = get_model()

{'iterations': 140, 'learning_rate': 0.1, 'random_seed': 144, 'custom_metric': 'F1', 'loss_function': 'Logloss', 'class_weights': [1, 20]}


In [171]:
# export
# r_itemcf, r_binn, r_itemcf_yl = load_match_items()
# dff = data_preporcess([r_itemcf, r_binn, r_itemcf_yl], match_num, phase)
# dff = fe(dff)
# train_data, val_data, cat_cols = get_model_input(dff)

r = model.fit(train_data, 
#               eval_set=val_data, 
              verbose=False,plot=False)
dff.loc[dff['phase']==6, 'fscore'] = model.predict_proba(val_data)[:, 1]

dff.loc[dff['phase']==6, 'frank'] = dff.loc[dff['phase']==6].groupby('user_id')['fscore'].rank(method='first', ascending=False)
r = val1(dff.loc[(dff['phase']==6)&(dff['frank']<101)], 100, all_click_df[6][1])

2020-06-03 14:55:11.593 | INFO     | __main__:val1:19 - recall50:0.14332784184514002
2020-06-03 14:55:11.596 | INFO     | __main__:val1:19 - recall100:0.21636463481603516


In [ ]:
2020-05-28 17:35:07.937 | INFO     | __main__:val1:24 - recall50:0.17298187808896212
2020-05-28 17:35:07.940 | INFO     | __main__:val1:24 - recall100:0.21087314662273476
                    
2020-06-03 11:54:35.829 | INFO     | __main__:val1:19 - recall50:0.2317408017572762
2020-06-03 11:54:35.835 | INFO     | __main__:val1:19 - recall100:0.3399231191652938
                    

In [162]:
for a in range(10):
    print(a)
    r = model.fit(train_data, 
#                   eval_set=val_data, 
                  verbose=False,plot=False, init_model=model)
    dff.loc[dff['phase']==6, 'fscore'] = model.predict_proba(val_data)[:, 1]

    dff.loc[dff['phase']==6, 'frank'] = dff.loc[dff['phase']==6].groupby('user_id')['fscore'].rank(method='first', ascending=False)

    r = val1(dff.loc[(dff['phase']==6)&(dff['frank']<101)], 100, all_click_df[6][1])

0


2020-06-03 13:59:22.175 | INFO     | __main__:val1:19 - recall50:0.13344316309719934
2020-06-03 13:59:22.179 | INFO     | __main__:val1:19 - recall100:0.21252059308072488


1


2020-06-03 14:00:58.664 | INFO     | __main__:val1:19 - recall50:0.13509060955518945
2020-06-03 14:00:58.667 | INFO     | __main__:val1:19 - recall100:0.21361889071938495


2


...

2020-06-03 14:10:26.913 | INFO     | __main__:val1:19 - recall50:0.1416803953871499
2020-06-03 14:10:26.916 | INFO     | __main__:val1:19 - recall100:0.21691378363536518


8


2020-06-03 14:12:03.078 | INFO     | __main__:val1:19 - recall50:0.1400329489291598
2020-06-03 14:12:03.082 | INFO     | __main__:val1:19 - recall100:0.21691378363536518


9


2020-06-03 14:13:37.451 | INFO     | __main__:val1:19 - recall50:0.13948380010982978
2020-06-03 14:13:37.454 | INFO     | __main__:val1:19 - recall100:0.2158154859967051


In [163]:
for a in range(10):
    print(a)
    r = model.fit(train_data, 
#                   eval_set=val_data, 
                  verbose=False,plot=False, init_model=model)
    dff.loc[dff['phase']==6, 'fscore'] = model.predict_proba(val_data)[:, 1]

    dff.loc[dff['phase']==6, 'frank'] = dff.loc[dff['phase']==6].groupby('user_id')['fscore'].rank(method='first', ascending=False)

    r = val1(dff.loc[(dff['phase']==6)&(dff['frank']<101)], 100, all_click_df[6][1])

0


2020-06-03 14:16:09.850 | INFO     | __main__:val1:19 - recall50:0.14058209774848984
2020-06-03 14:16:09.853 | INFO     | __main__:val1:19 - recall100:0.21636463481603516


1


2020-06-03 14:17:44.306 | INFO     | __main__:val1:19 - recall50:0.14222954420647996
2020-06-03 14:17:44.309 | INFO     | __main__:val1:19 - recall100:0.21801208127402527


2


...

2020-06-03 14:27:21.452 | INFO     | __main__:val1:19 - recall50:0.1411312465678199
2020-06-03 14:27:21.455 | INFO     | __main__:val1:19 - recall100:0.21526633717737506


8


2020-06-03 14:28:56.973 | INFO     | __main__:val1:19 - recall50:0.1411312465678199
2020-06-03 14:28:56.976 | INFO     | __main__:val1:19 - recall100:0.2158154859967051


9


2020-06-03 14:30:32.268 | INFO     | __main__:val1:19 - recall50:0.1400329489291598
2020-06-03 14:30:32.272 | INFO     | __main__:val1:19 - recall100:0.21526633717737506


In [ ]:
for a in range(10):
    print(a)
    r = model.fit(train_data, 
#                   eval_set=val_data, 
                  verbose=False,plot=False, init_model=model)
    dff.loc[dff['phase']==6, 'fscore'] = model.predict_proba(val_data)[:, 1]

    dff.loc[dff['phase']==6, 'frank'] = dff.loc[dff['phase']==6].groupby('user_id')['fscore'].rank(method='first', ascending=False)

    r = val1(dff.loc[(dff['phase']==6)&(dff['frank']<101)], 100, all_click_df[6][1])

0


2020-06-03 14:33:42.404 | INFO     | __main__:val1:19 - recall50:0.13893465129049973
2020-06-03 14:33:42.407 | INFO     | __main__:val1:19 - recall100:0.21361889071938495


1


2020-06-03 14:35:17.969 | INFO     | __main__:val1:19 - recall50:0.1400329489291598
2020-06-03 14:35:17.984 | INFO     | __main__:val1:19 - recall100:0.214168039538715


2


...

4


2020-06-03 14:40:03.252 | INFO     | __main__:val1:19 - recall50:0.13948380010982978
2020-06-03 14:40:03.255 | INFO     | __main__:val1:19 - recall100:0.214168039538715


5


2020-06-03 14:41:39.228 | INFO     | __main__:val1:19 - recall50:0.13893465129049973
2020-06-03 14:41:39.232 | INFO     | __main__:val1:19 - recall100:0.21471718835804504


6


In [172]:
# model.save_model(os.path.join(args.DATA_DIR, 'data_gen/catboost_model500_feat_last123_all.dump'))

# predict

In [9]:
item_feat = get_item_data()

item_feat['text_vec'] = item_feat.iloc[:, 1:129].values.tolist()
item_feat['img_vec'] = item_feat.iloc[:, 129:257].values.tolist()

item_feat['text_vec'] = item_feat['text_vec'].map(np.array)

item_feat['img_vec'] = item_feat['img_vec'].map(np.array)

item_feat.set_index('item_id', inplace=True)

/root/data1/myappsexlib/a_0330_0527_Modern_E_Commerce_Platform_Debiasing/code/eda.py:32: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  converters={'item_id':str})


In [118]:
r_itemcf = PickleWrapper.loadFromFile(os.path.join(args.DATA_DIR, 'data_gen/r_list_itemcf_test_0527.pkl'))

r_binn = PickleWrapper.loadFromFile(os.path.join(args.DATA_DIR, 'data_gen/r_list_binn_test_0527.pkl'))

r_itemcf_yl = PickleWrapper.loadFromFile(os.path.join(args.DATA_DIR, 'data_gen/r_list_itemcf_yulao_test_0527.pkl'))
r_list_w2v = PickleWrapper.loadFromFile(os.path.join(args.DATA_DIR, 'data_gen/r_list_w2v_test_0603_all.pkl'))

In [120]:
phase = [0, 1, 2, 3, 4, 5, 6]
match_num = 500

df = data_preporcess([r_itemcf, r_binn, r_itemcf_yl, r_list_w2v], match_num, phase, mode='predict')


merge Multi-channel recall...




  0%|          | 0/7 [00:00<?, ?it/s]

 14%|█▍        | 1/7 [02:08<12:51, 128.56s/it]

 29%|██▊       | 2/7 [04:22<10:50, 130.19s/it]

 43%|████▎     | 3/7 [06:36<08:45, 131.28s/it]

 57%|█████▋    | 4/7 [08:50<06:36, 132.18s/it]

 71%|███████▏  | 5/7 [11:01<04:23, 131.68s/it]

 86%|████████▌ | 6/7 [13:09<02:10, 130.80s/it]

100%|██████████| 7/7 [15:15<00:00, 130.76s/it]


In [121]:
df.shape

(15530154, 15)

In [133]:
df['t'] = range(df.shape[0])

In [124]:
df.head(2)

,user_id,item_id_pred,score_x,rank_x,phase,score_y,rank_y,score_x,rank_x,score_y,rank_y,item_cnt,last_1,last_2,last_3,item_id_pred_text_vec,item_id_pred_img_vec
0,11,10528,9.343545,1.0,0,0.749194,21.0,0.110274,2.0,34.100440,4.0,7.0,28210,64689,20317,"[3.5714921951293945, -3.301568031311035, -1.44...","[-2.7678756713867188, -0.804641842842102, -2.3..."
1,11,79868,6.949118,2.0,0,2.023002,1.0,0.007360,322.0,10.992932,454.0,47.0,28210,64689,20317,"[2.3873836994171143, 1.274107813835144, 0.2217...","[-2.599696159362793, 1.5488953590393066, -1.48..."


In [128]:
temp_ = pd.read_pickle(os.path.join(args.DATA_DIR, f'data_gen/dft{match_num}_test_features.pkl'))

In [129]:
temp_.columns

Index(['user_id', 'item_id_pred', 'phase', 'score_x', 'rank_x', 'score_y',
       'rank_y', 'score', 'rank', 'item_cnt', 'sim1_text', 'sim1_img',
       'sim2_text', 'sim2_img', 'sim3_text', 'sim3_img', 'sim4_text',
       'sim4_img', 'sim5_text', 'sim5_img', 'rank1_sub_rank2',
       'rank1_add_rank2', 'rank1_mul_rank2', 'rank1_sub_rank3',
       'rank1_add_rank3', 'rank1_mul_rank3', 'rank2_sub_rank3',
       'rank2_add_rank3', 'rank2_mul_rank3', 'score1_sub_score2',
       'score1_add_score2', 'score1_mul_score2', 'score1_sub_score3',
       'score1_add_score3', 'score1_mul_score3', 'score2_sub_score3',
       'score2_add_score3', 'score2_mul_score3', 'sim1_text_img',
       'sim2_text_img', 'sim3_text_img', 'sim4_text_img', 'sim5_text_img',
       'sim12_text', 'sim123_text', 'sim1234_text', 'sim12345_text',
       'sim12_img', 'sim123_img', 'sim1234_img', 'sim12345_img',
       'sim12_text_img', 'sim123_text_img', 'sim1234_text_img',
       'sim12345_text_img'],
      dtype='object

In [134]:
df_ = pd.merge(df, temp_[['user_id', 'item_id_pred', 'phase', 'sim1_text', 'sim1_img',
       'sim2_text', 'sim2_img', 'sim3_text', 'sim3_img']], on=['user_id', 'item_id_pred', 'phase'])

In [132]:
df_.shape

(11168101, 23)

In [135]:
df1 = df[~df.t.isin(df_.t)]

In [136]:
df1.shape

(4362053, 18)

In [137]:
df1.head(2)

,user_id,item_id_pred,score_x,rank_x,phase,score_y,rank_y,score_x,rank_x,score_y,rank_y,item_cnt,last_1,last_2,last_3,item_id_pred_text_vec,item_id_pred_img_vec,t
875,11,81221,0.0,0.0,0,0.0,0.0,0.0,0.0,39.036631,2.0,5.0,28210,64689,20317,"[2.7964396476745605, -0.6318608522415161, 2.20...","[-1.5980700254440308, 2.5623795986175537, 1.70...",875
876,11,11679,0.0,0.0,0,0.0,0.0,0.0,0.0,29.522777,7.0,4.0,28210,64689,20317,"[-0.25924938917160034, 1.6548818349838257, 0.5...","[0.8699896335601807, -4.748091220855713, -0.07...",876


In [138]:
# df1['item_id_pred_text_vec'] = df1.item_id_pred.map(lambda x: item_feat['text_vec'][x] if x in item_feat['text_vec'] else np.zeros(128))

# df1['item_id_pred_img_vec'] = df1.item_id_pred.map(lambda x: item_feat['img_vec'][x] if x in item_feat['img_vec'] else np.arange(128))

for c in tqdm('last_1 last_2 last_3'.split()):
    df1[f'{c}_text_vec'] = df1.item_id_pred.map(lambda x: item_feat['text_vec'][x] if x in item_feat['text_vec'] else np.zeros(128))

    df1[f'{c}_img_vec'] = df1.item_id_pred.map(lambda x: item_feat['img_vec'][x] if x in item_feat['img_vec'] else np.arange(128))


df1['sim1_text'] = (df1['item_id_pred_text_vec'] * df1['last_1_text_vec']).map(sum)

df1['sim1_img'] = (df1['item_id_pred_img_vec'] * df1['last_1_img_vec']).map(sum)

df1['sim2_text'] = (df1['item_id_pred_text_vec'] * df1['last_2_text_vec']).map(sum)
df1['sim2_img'] = (df1['item_id_pred_img_vec'] * df1['last_2_img_vec']).map(sum)

df1['sim3_text'] = (df1['item_id_pred_text_vec'] * df1['last_3_text_vec']).map(sum)
df1['sim3_img'] = (df1['item_id_pred_img_vec'] * df1['last_3_img_vec']).map(sum)




  0%|          | 0/3 [00:00<?, ?it/s]/root/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/root/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  



 33%|███▎      | 1/3 [03:17<06:34, 197.06s/it]


 67%|██████▋   | 2/3 [06:39<03:18, 198.74s/it]


100%|██████████| 3/3 [10:01<00:00, 200.37s/it]
/root/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 

In [142]:
df1.columns = ['user_id', 'item_id_pred', 'score_1', 'rank_1', 'phase', 'score_2',
       'rank_2', 'score_3', 'rank_3', 'score_4', 'rank_4', 'item_cnt',
       'last_1', 'last_2', 'last_3', 'item_id_pred_text_vec',
       'item_id_pred_img_vec', 't', 'last_1_text_vec', 'last_1_img_vec',
       'last_2_text_vec', 'last_2_img_vec', 'last_3_text_vec',
       'last_3_img_vec', 'sim1_text', 'sim1_img', 'sim2_text', 'sim2_img',
       'sim3_text', 'sim3_img']

In [143]:
df_.columns = ['user_id', 'item_id_pred', 'score_1', 'rank_1', 'phase', 'score_2',
       'rank_2', 'score_3', 'rank_3', 'score_4', 'rank_4', 'item_cnt',
       'last_1', 'last_2', 'last_3', 'item_id_pred_text_vec',
       'item_id_pred_img_vec', 't', 'sim1_text', 'sim1_img', 'sim2_text',
       'sim2_img', 'sim3_text', 'sim3_img']

In [144]:
dff = df1[df_.columns].append(df_)

In [145]:
dff.shape

(15530154, 24)

In [146]:
dff = fe(dff)





  0%|          | 0/2 [00:00<?, ?it/s]


 50%|█████     | 1/2 [00:00<00:00,  2.19it/s]


100%|██████████| 2/2 [00:00<00:00,  2.29it/s]


In [147]:
dff.columns

Index(['user_id', 'item_id_pred', 'score_1', 'rank_1', 'phase', 'score_2',
       'rank_2', 'score_3', 'rank_3', 'score_4', 'rank_4', 'item_cnt',
       'last_1', 'last_2', 'last_3', 'item_id_pred_text_vec',
       'item_id_pred_img_vec', 't', 'sim1_text', 'sim1_img', 'sim2_text',
       'sim2_img', 'sim3_text', 'sim3_img', 'rank1_sub_rank2',
       'rank1_add_rank2', 'rank1_mul_rank2', 'rank1_sub_rank3',
       'rank1_add_rank3', 'rank1_mul_rank3', 'rank1_sub_rank4',
       'rank1_add_rank4', 'rank1_mul_rank4', 'rank2_sub_rank3',
       'rank2_add_rank3', 'rank2_mul_rank3', 'rank2_sub_rank4',
       'rank2_add_rank4', 'rank2_mul_rank4', 'rank3_sub_rank4',
       'rank3_add_rank4', 'rank3_mul_rank4', 'score1_sub_score2',
       'score1_add_score2', 'score1_mul_score2', 'score1_sub_score3',
       'score1_add_score3', 'score1_mul_score3', 'score1_sub_score4',
       'score1_add_score4', 'score1_mul_score4', 'score2_sub_score3',
       'score2_add_score3', 'score2_mul_score3', 'score2_su

In [148]:
cols = ['user_id', 'item_id_pred', 'score_1', 'rank_1', 'phase', 'score_2',
       'rank_2', 'score_3', 'rank_3', 'score_4', 'rank_4', 'item_cnt',
       'last_1', 'last_2', 'last_3', 'sim1_text', 'sim1_img', 'sim2_text',
       'sim2_img', 'sim3_text', 'sim3_img', 'rank1_sub_rank2',
       'rank1_add_rank2', 'rank1_mul_rank2', 'rank1_sub_rank3',
       'rank1_add_rank3', 'rank1_mul_rank3', 'rank1_sub_rank4',
       'rank1_add_rank4', 'rank1_mul_rank4', 'rank2_sub_rank3',
       'rank2_add_rank3', 'rank2_mul_rank3', 'rank2_sub_rank4',
       'rank2_add_rank4', 'rank2_mul_rank4', 'rank3_sub_rank4',
       'rank3_add_rank4', 'rank3_mul_rank4', 'score1_sub_score2',
       'score1_add_score2', 'score1_mul_score2', 'score1_sub_score3',
       'score1_add_score3', 'score1_mul_score3', 'score1_sub_score4',
       'score1_add_score4', 'score1_mul_score4', 'score2_sub_score3',
       'score2_add_score3', 'score2_mul_score3', 'score2_sub_score4',
       'score2_add_score4', 'score2_mul_score4', 'score3_sub_score4',
       'score3_add_score4', 'score3_mul_score4', 'sim1_text_img',
       'sim2_text_img', 'sim3_text_img', 'sim12_text', 'sim123_text',
       'sim12_img', 'sim123_img', 'sim12_text_img', 'sim123_text_img']
# dff[cols].to_pickle(os.path.join(args.DATA_DIR, f'data_gen/dft{match_num}_test_features_all.pkl'))

In [173]:
dff = pd.read_pickle(os.path.join(args.DATA_DIR, f'data_gen/dft{match_num}_test_features_all.pkl'))

In [176]:
test_data, cat_cols = get_model_input(dff, mode='predict')


In [19]:
# set(df.user_id)-set(dft.user_id) # {'14997'}

{'14997'}

In [177]:
from catboost import Pool, CatBoostClassifier

In [178]:
model = CatBoostClassifier()
model.load_model(os.path.join(args.DATA_DIR, 'data_gen/catboost_model500_feat_last123_all.dump'))

In [179]:
dff.loc[:, 'fscore'] = model.predict_proba(test_data)[:, 1]

In [182]:
dff.loc[:, 'frank'] = dff.groupby('user_id')['fscore'].rank(method='first', ascending=False)

In [186]:
dff.sort_values('user_id frank'.split()).head(20)

,user_id,item_id_pred,score_1,rank_1,phase,score_2,rank_2,score_3,rank_3,score_4,rank_4,item_cnt,last_1,last_2,last_3,sim1_text,sim1_img,sim2_text,sim2_img,sim3_text,sim3_img,rank1_sub_rank2,rank1_add_rank2,rank1_mul_rank2,rank1_sub_rank3,rank1_add_rank3,rank1_mul_rank3,rank1_sub_rank4,rank1_add_rank4,rank1_mul_rank4,rank2_sub_rank3,rank2_add_rank3,rank2_mul_rank3,rank2_sub_rank4,rank2_add_rank4,rank2_mul_rank4,rank3_sub_rank4,rank3_add_rank4,rank3_mul_rank4,score1_sub_score2,score1_add_score2,score1_mul_score2,score1_sub_score3,score1_add_score3,score1_mul_score3,score1_sub_score4,score1_add_score4,score1_mul_score4,score2_sub_score3,score2_add_score3,score2_mul_score3,score2_sub_score4,score2_add_score4,score2_mul_score4,score3_sub_score4,score3_add_score4,score3_mul_score4,sim1_text_img,sim2_text_img,sim3_text_img,sim12_text,sim123_text,sim12_img,sim123_img,sim12_text_img,sim123_text_img,fscore,frank
1281712,1,32360,6.932601,1.0,1,0.720665,1.0,0.118531,1.0,11.650906,80.0,4.0,72939,17887,31443,863.692091,565.325717,863.692091,565.325717,863.692091,565.325717,0.0,2.0,1.0,0.0,2.0,1.0,-79.0,81.0,80.0,0.0,2.0,1.0,-79.0,81.0,80.0,-79.0,81.0,80.0,6.211936,7.653266,4.996081,6.814071,7.051132,0.821726,-4.718305,18.583507,80.771086,0.602134,0.839195,0.085421,-10.930242,12.371571,8.396396,-11.532376,11.769437,1.380989,488267.350695,488267.350695,488267.350695,1727.384182,2591.076272,1130.651434,1695.977152,9.765347e+05,1.464802e+06,0.199372,1.0
1281716,1,46297,3.534822,5.0,1,0.313277,56.0,0.073649,4.0,10.195960,129.0,4.0,72939,17887,31443,806.520190,560.152918,806.520190,560.152918,806.520190,560.152918,-51.0,61.0,280.0,1.0,9.0,20.0,-124.0,134.0,645.0,52.0,60.0,224.0,-73.0,185.0,7224.0,-125.0,133.0,516.0,3.221544,3.848099,1.107379,3.461172,3.608471,0.260337,-6.661138,13.730781,36.040898,0.239628,0.386927,0.023073,-9.882682,10.509237,3.194162,-10.122310,10.269609,0.750925,451774.637925,451774.637925,451774.637925,1613.040379,2419.560569,1120.305837,1680.458755,9.035493e+05,1.355324e+06,0.103043,2.0
1281719,1,87837,3.241958,8.0,1,0.536789,5.0,0.038941,33.0,0.000000,0.0,12.0,72939,17887,31443,1044.191308,526.457017,1044.191308,526.457017,1044.191308,526.457017,3.0,13.0,40.0,-25.0,41.0,264.0,8.0,8.0,0.0,-28.0,38.0,165.0,5.0,5.0,0.0,33.0,33.0,0.0,2.705169,3.778747,1.740247,3.203017,3.280899,0.126244,3.241958,3.241958,0.000000,0.497848,0.575730,0.020903,0.536789,0.536789,0.000000,0.038941,0.038941,0.000000,549721.840946,549721.840946,549721.840946,2088.382617,3132.573925,1052.914033,1579.371050,1.099444e+06,1.649166e+06,0.065992,3.0
1281713,1,100116,4.453803,2.0,1,0.363821,35.0,0.067410,6.0,9.035917,214.0,4.0,72939,17887,31443,1292.133060,726.512387,1292.133060,726.512387,1292.133060,726.512387,-33.0,37.0,70.0,-4.0,8.0,12.0,-212.0,216.0,428.0,29.0,41.0,210.0,-179.0,249.0,7490.0,-208.0,220.0,1284.0,4.089982,4.817625,1.620389,4.386393,4.521213,0.300231,-4.582114,13.489721,40.244200,0.296411,0.431231,0.024525,-8.672096,9.399739,3.287460,-8.968507,9.103327,0.609112,938750.674026,938750.674026,938750.674026,2584.266120,3876.399179,1453.024775,2179.537162,1.877501e+06,2.816252e+06,0.065171,4.0
1281717,1,91290,3.522240,6.0,1,0.536789,7.0,0.059625,10.0,7.198735,412.0,6.0,72939,17887,31443,918.670205,452.535851,918.670205,452.535851,918.670205,452.535851,-1.0,13.0,42.0,-4.0,16.0,60.0,-406.0,418.0,2472.0,-3.0,17.0,70.0,-405.0,419.0,2884.0,-402.0,422.0,4120.0,2.985451,4.059028,1.890699,3.462615,3.581864,0.210012,-3.676496,10.720975,25.355671,0.477164,0.596414,0.032006,-6.661947,7.735524,3.864201,-7.139111,7.258360,0.429222,415731.203055,415731.203055,415731.203055,1837.340410,2756.010616,905.071702,1357.607553,8.314624e+05,1.247194e+06,0.058183,5.0
1281720,1,103887,3.166490,9.0,1,0.407387,17.0,0.021934,71.0,9.971464,137.0,3.0,72939,17887,31443,189.196155,453.151189,189.196155,453.151189,189.196155,453.151189,-8.0,26.0,153.0,-62.0,80.0,639.0,-128.0,146.0,1233.0,-54.0,88.0,1207.0,-120.0,154.0,2329.0,-66.0,208.0,9727.0,2.759103,3.573877,1.289988,3.144556

In [187]:
df1 = dff.sort_values(['user_id', 'fscore'], ascending=False)

In [188]:
sub = df1[df1.frank<51]

In [189]:
sub = sub.groupby('user_id')['item_id_pred'].agg(list)

In [190]:
sub

user_id
1        [32360, 46297, 87837, 100116, 91290, 103887, 1...
100      [100089, 112676, 57797, 13337, 27324, 17927, 4...
10001    [27822, 45632, 79132, 110920, 109849, 25257, 4...
10002    [100447, 92204, 74285, 80136, 5525, 68539, 467...
10003    [91369, 94839, 10153, 30879, 64611, 61748, 900...
                               ...                        
9989     [107329, 111222, 101475, 80227, 107960, 107968...
9991     [14239, 20258, 115035, 17905, 44549, 18287, 69...
9992     [42102, 11182, 36448, 18277, 39725, 64053, 309...
9993     [13354, 80129, 62601, 66514, 78046, 36207, 161...
9994     [67789, 71249, 6973, 43622, 14347, 5194, 46996...
Name: item_id_pred, Length: 12080, dtype: object

In [191]:
sub1 = pd.DataFrame()

for i in range(50):
    sub1[f'c{i}'] = sub.str.get(i)

sub1.applymap(int).to_csv('baseline1_itemcf36666.csv', header=False)

In [192]:
sub1.shape

(12080, 50)